In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import itertools as it
from collections import Counter, defaultdict, deque
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
import itertools
import copy
sns.set_style('white')
sns.set_context('notebook', font_scale=1.3)

from agents import Agent
from evaluation import get_util
# from model_utils import *

In [2]:
from mouselab import MouselabEnv
from distributions import Categorical, Normal

def make_envs(cost=1.00, n=100, seed=None,variance_structure="constant_high",branching=[4,1,2]):
    if seed is not None:
        np.random.seed(seed)
    
    depth = len(branching)
    
    if variance_structure is "constant_high":
        sigmas = np.concatenate( (np.array([0]),20*np.ones(depth)))
    if variance_structure is "increasing":
        sigmas = [0, 2, 4, 20]
    if variance_structure is "decreasing":
        sigmas = [0,20,4,2]
    if variance_structure is "constant_low":
        sigmas = np.concatenate( (np.array([0]),3*np.ones(depth)))
        
    def reward(depth):
        if depth > 0:
            return Normal(0, sigmas[depth]).to_discrete(6)
        return 0.

    envs = [MouselabEnv.new_symmetric(branching, reward)
            for _ in range(n)]
    for env in envs:
        env.cost=-cost
    
    return envs

In [3]:
cost = 1
envs = make_envs(cost,10,None,"increasing",branching=[2,2])
env = envs[0]

In [4]:
env.paths

[[1, 2], [1, 3], [4, 5], [4, 6]]

In [5]:
env._state

(0, Cat, Cat, Cat, Cat, Cat, Cat)

In [6]:
env.tree

[[1, 4], [2, 3], [], [], [5, 6], [], []]

In [7]:
# fancy initialization function, but doesn't seem necessary
# simpler version built into the class
def build_path(r):   
    if env.tree[r] == []:
        return ([[]], [0])
    paths = []
    path_moves = []
    for n in env.tree[r]:
        new_paths, new_path_moves = build_path(n)
        for i in range(len(new_paths)):
            new_paths[i].insert(0,n)
            new_move = int(hasattr(env._state[n],'sample'))
            path_moves.append(new_path_moves[i]+new_move)
            paths.append(new_paths[i])
    return (paths,path_moves)

In [8]:
def option_util(x,sigma):
    return sigma*scipy.stats.norm.pdf(x/sigma) - np.abs(x)*scipy.stats.norm.cdf(-np.abs(x)/sigma)

In [9]:
def get_all_options(env):
    paths = env.paths #list of all paths
    avail_moves = [0,]*len(paths) #list of moves available in each path
    path_obs = [] #value of observed nodes in each path
    path_nodes = [] #the unobserved nodes of each path
    path_stds = [] #the std deviation of the unobserved nodes of each path
    
    options = [] #list of all options
    option_utils = [] #list of the utility of each option
    
    for i in range(len(paths)):
        stds = []
        nodes = []
        obs = 0
        
        for node in paths[i]:
            if hasattr(env._state[node],'sample'):
                stds.append(env._state[node].var())
                nodes.append(node)
                avail_moves[i] += 1
            else:
                obs += env._state[node]
                
        path_obs.append(obs)
        path_stds.append(stds)
        path_nodes.append(nodes)
        
        for j in range(avail_moves[i]):
            options.append((i,j+1))
    max_obs = np.max(path_obs)
    
    for option in options:
        path, obs = option
        option_utils.append(option_util(path_obs[path]-max_obs,np.sqrt(np.sum(path_stds[path][:obs]))) + obs*env.cost)
    
    return options, option_utils, path_nodes, path_stds, path_obs, avail_moves

In [10]:
def pick_option_moves(env):
    options, option_utils, path_nodes, path_stds, path_obs, avail_moves = get_all_options(env)
    
    #c is for chosen
    cpath, cobs = options[np.random.choice(np.arange(len(options))[option_utils == np.max(option_utils)])]
    cpath_stds = np.array(path_stds[cpath])[:cobs]
    cpath_nodes = np.array(path_nodes[cpath])[:cobs]
    b = np.random.random(cpath_nodes.size)
    
    return cpath_nodes[np.lexsort((b,cpath_stds))]

In [11]:
import time
t = time.process_time()
for i in range(1000):
#     print(get_options(env))
    pick_option_moves(env)
elapsed =time.process_time() - t 
print(elapsed)

8.347992847


In [12]:
def all_option_insts(path_nodes,path_stds,n_obs):
    insts = [[]]
    n_remaining_obs = n_obs

    vals, inverse, count = np.unique(path_stds, return_inverse=True,
                                  return_counts=True)
    rows, cols = np.where(inverse == np.arange(len(vals))[:, np.newaxis])
    _, inverse_rows = np.unique(rows, return_index=True)
    res = np.split(cols, inverse_rows[1:])

    for i in range(len(res)):
        new_insts = []

        n_new_nodes = len(res[-i-1])
        if n_new_nodes < n_remaining_obs:
            n_remaining_obs -= n_new_nodes
        else:
            n_new_nodes = n_remaining_obs
            n_remaining_obs = 0  

        for new_nodes in itertools.permutations(res[-i-1],n_new_nodes):
            for inst in insts:
                new_insts.append(inst + list(np.array(path_nodes)[list(new_nodes)]))
        insts = new_insts
        if n_remaining_obs == 0:
            break
            
    return insts

In [13]:
path_stds= [10.00,10.00,9,8.0,8,7,7,6]
path_nodes = np.arange(len(path_stds))
all_option_insts(path_nodes,path_stds,6)

[[0, 1, 2, 3, 4, 5],
 [1, 0, 2, 3, 4, 5],
 [0, 1, 2, 4, 3, 5],
 [1, 0, 2, 4, 3, 5],
 [0, 1, 2, 3, 4, 6],
 [1, 0, 2, 3, 4, 6],
 [0, 1, 2, 4, 3, 6],
 [1, 0, 2, 4, 3, 6]]

The following equation gives the likelihood of a click sequence in this model:

$$ l(C) = \sum_{O \in \mathcal{O}(C)} l(O)$$

where $C$ is a click sequence, and $\mathcal{O}(C)$ is the set of option sequences that $C$ could parse into. To get the likelihood of an option sequence, we use the following equation:

$$ l(O) = \prod_{o \in O} \mathbb{P}_{DC}(o) $$

where $\mathbb{P}_{DC}(o)$ is the probability of a given option in our model. We assume a generative model that picks a random move with probability $p_r$, or otherwise picks from one of the available options under the directed cognition model. This gives us the following equation: 

$$\mathbb{P}_{DC}(o) = (1-p_r)*s(o)+p_r*\alpha$$ 

$s(o)$ is the softmax probability of our option among all available directed cognition options, which, for a given temperature parameter $t = \frac{1}{\beta}$ is equal to:

$$ s(o)= \frac{e^{\beta v(o)}}{\sum_{o'} e^{\beta v(o')}} $$

where $v(o)$ is the value of an option using the directed cognition model. We define $\alpha$ as the probability that an option in the sequence would have been generated by the error process, and it is defined as:

$$\alpha = \prod_{k = 1}^ \text{length(o)} \frac{1}{n_{ac}-k+1}$$

where $n_{ac}$ is the number of available clicks.

In [78]:
def parse_options(env,click_sequence,t=1,p_rand=0.0001):
    if click_sequence == []:
        return True, [[]], [1]
    option_insts = dict() #list of all possible option instantiations
    option_seqs = []
    likelihoods = []
    done = False
    
    paths = env.paths
    options, option_utils, path_nodes, path_stds, path_obs, avail_moves = get_all_options(env)
    
    for option in options:
        path, obs = option
        option_insts[option] = all_option_insts(path_nodes[path],path_stds[path],obs)
        
    #single click options
    sc_opt = (-1,1)
    options.append(sc_opt)
    option_utils.append(-np.inf)
    option_insts[sc_opt] = [[a] for a in env.actions(env._state)]
    n_available_clicks = len(option_insts[sc_opt])-1
        
    for i in range(1,min(len(paths[0]),len(click_sequence))+1):  
        for j in range(len(options)):
            option = options[j]
            for inst in option_insts[option]:      
                if np.array_equal(click_sequence[:i],inst):
                    copy_env = copy.deepcopy(env)                   
                    for a in click_sequence[:i]:
                        copy_env._step(a)
                        
                    will_done, remaining, rem_likelihoods = parse_options(copy_env,click_sequence[i:])
                    done = done or will_done
                    
                    if done:
                        for k in range(len(remaining)): 
                            latter = remaining[k]
                            option_seqs.append([option]+latter)
                            
                            l_opt_seq = (1-p_rand)*np.exp(1/t*option_utils[j])/np.sum(1/t*np.exp(option_utils))
                            l_opt_seq += p_rand*np.prod([1/(n_available_clicks-k+1) for k in range(1,option[1]+1)])
                            l_opt_seq *= rem_likelihoods[k]
                            
                            likelihoods.append(l_opt_seq)
    
    return done, option_seqs, likelihoods

In [15]:
cost = 1
envs = make_envs(cost,10,None,"increasing",branching=[2,2])
env = envs[0]

In [16]:
list(env.actions(env._state))

[1, 2, 3, 4, 5, 6, 7]

In [17]:
click_seq = [1,4,2,3]
done,option_seqs,likelihoods = parse_options(env,click_seq)
option_seqs

[[(-1, 1), (-1, 1), (0, 1), (1, 1)],
 [(-1, 1), (-1, 1), (0, 1), (-1, 1)],
 [(-1, 1), (-1, 1), (-1, 1), (1, 1)],
 [(-1, 1), (-1, 1), (-1, 1), (-1, 1)]]

In [18]:
np.exp(np.log(np.sum(likelihoods))/len(click_seq))

0.0022958636312342242

In [19]:
click_seqs = [[i] for i in range(1,7)]
ls = []
for click_seq in click_seqs:
    done,option_seqs,likelihoods = parse_options(env,click_seq)
    print(likelihoods)
    ls.append(np.exp(np.log(np.sum(likelihoods))/len(click_seq)))
print(ls)

[1.6666666666666667e-05]
[0.12639260279504533, 1.6666666666666667e-05]
[0.12639260279504533, 1.6666666666666667e-05]
[1.6666666666666667e-05]
[0.12639260279504533, 1.6666666666666667e-05]
[0.12639260279504533, 1.6666666666666667e-05]
[1.6666666666666661e-05, 0.12640926946171199, 0.12640926946171199, 1.6666666666666661e-05, 0.12640926946171199, 0.12640926946171199]


In [20]:
np.sum(ls)

0.5056704111801813

In [21]:
def dc_model_ave(env,click_seq,t=1,p_rand=0.0001):
    done,option_seqs,likelihoods = parse_options(env,click_seq,t,p_rand)
    return np.exp(np.log(np.sum(likelihoods))/len(click_seq))

In [22]:
dc_model_ave(env,click_seq)

0.12640926946171199

# Modeling

In [23]:
def make_env(mu=0, sigma=5, quantization=4, cost=1.00, seed=None, branching=[3,1,2], **kwargs):
    if seed is not None:
        np.random.seed(seed)

    def reward(depth):
        if depth > 0:
            x = np.array([-2,-1,1,2])
            return Categorical(mu + sigma * x)
        return 0.

    return MouselabEnv.new_symmetric(branching, reward, cost=cost, **kwargs)

env = make_env(ground_truth=False)

In [24]:
from analysis_utils import *
VERSION = 'c1.1'
exp_data = get_data(VERSION, '../experiment/data')

pdf = exp_data['participants']
pdf = pdf.loc[pdf.completed].copy()
print(f'{len(pdf)} participants')
complete = list(pdf.index)

def extract(q):
    return list(map(int, q['click']['state']['target']))

mdf = exp_data['mouselab-mdp'].query('pid == @complete').copy()
mdf['clicks'] = mdf.queries.apply(extract)
mdf['n_clicks'] = mdf.clicks.apply(len)
mdf['thinking'] = mdf['rt'].apply(get(0, default=0))

tdf = mdf.query('block == "test"').copy()
tdf.trial_index -= tdf.trial_index.min()
tdf.trial_index = tdf.trial_index.astype(int)
tdf.trial_id = tdf.trial_id.astype(int)

# pdf['total_time'] = exp_data['survey'].time_elapsed / 60000

pdf['n_clicks'] = tdf.groupby('pid').n_clicks.mean()
pdf['score'] = tdf.groupby('pid').score.mean()
pdf['thinking'] = mdf.groupby('pid').thinking.mean()

Error importing rpy2
60 participants


In [25]:
import json
def excluded_pids():
    sdf = exp_data['survey-multi-choice'].query('pid == @complete').copy()
    sdf = pd.DataFrame(list(sdf.responses), index=sdf.index)
    correct = pd.Series(['-$10 to $10', '$1', '1 cent for every $1 you make in the game'])
    fail_quiz = (sdf != correct).sum(axis=1) > 1
    no_click = mdf.query('block == "train_inspector"').groupby('pid').n_clicks.sum() == 0
    return fail_quiz | no_click

exclude = excluded_pids()
tdf['exclude'] = list(exclude.loc[tdf.pid])
tdf = tdf.query('~exclude').copy().drop('exclude', axis=1)
print(f'excluding {exclude.sum()} out of {len(exclude)} partipicants')

excluding 9 out of 60 partipicants


In [26]:
def get_env(state_rewards):
    state_rewards[0] = 0
    return make_env(ground_truth=state_rewards)
tdf['env'] = tdf.state_rewards.apply(get_env)

# assert len(tdf.pid.unique()) == tdf.pid.max() + 1
def expand(df):
    for i, row in df.iterrows():
        env = row.env
        env.reset()
        for a in [*row.clicks, env.term_action]:
            yield {'pid': row.pid, 'trial_index': row.trial_index, 'trial_id': row.trial_id,
                   'state': env._state, 'action': a}
            env.step(a)

data = pd.DataFrame(expand(tdf))
print(f'{len(data)} observations)')
data.head()

7606 observations)


,action,pid,state,trial_id,trial_index
0,9,1,"(0, ((-10, 0.25), (-5, 0.25), (5, 0.25), (10, ...",8034619116489218048,0
1,13,1,"(0, ((-10, 0.25), (-5, 0.25), (5, 0.25), (10, ...",8034619116489218048,0
2,5,1,"(0, ((-10, 0.25), (-5, 0.25), (5, 0.25), (10, ...",38024071126860576,1
3,9,1,"(0, ((-10, 0.25), (-5, 0.25), (5, 0.25), (10, ...",38024071126860576,1
4,13,1,"(0, ((-10, 0.25), (-5, 0.25), (5, 0.25), (10, ...",38024071126860576,1


In [27]:
p_errs = np.linspace(0.01,0.25, 25)
temp = np.logspace(-5,1, 50)

In [70]:
def dc_likelihood_grid(env, click_seq, temp=np.logspace(-5,1, 50), p_error=np.linspace(0.01,0.25, 25)):
    n_temps = len(temp)
    n_p_errs = len(p_error)
    grid = np.zeros((n_temps,n_p_errs))
    for i in range(n_temps):
        for j in range(n_p_errs):
            grid[i,j] = np.log(np.sum(parse_options(env,click_seq,temp[i],p_error[j])[2]))
    return grid

In [62]:
list(tdf.iterrows())[0]

(90, action_times                                   [11467, 13434, 16274]
 actions                                             [left, left, up]
 block                                                           test
 path                                                  [0, 9, 10, 11]
 queries            {'mouseover': {'state': {'target': [], 'time':...
 rewards                                                  [5, -5, -5]
 rt                                               [11465, 1450, 2324]
 score                                                             -6
 simulation_mode                                   [None, None, None]
 state_rewards       [0, 5, -10, -5, 10, 10, 5, -10, 5, 5, -5, -5, 5]
 time_elapsed                                                  728336
 trial_index                                                        0
 trial_time                                                     20130
 trial_id                                         8034619116489218048
 trial_type     

In [79]:
do = 0
for i, row in tdf.iterrows():
    env = row.env
    env.reset()
    print(row.path[1:])
    click_seq = list(map(int, row.path[1:]))
    print(click_seq)
    grid = dc_likelihood_grid(env, click_seq)
    do += 1
    if do >= 1:
        break

#     print({'pid': row.pid, 'trial_index': row.trial_index, 'trial_id': row.trial_id,
#            'likelihood':dc_likelihood_grid(env, row.path[1:])})
#     mle = dc_maximum_likelihood(pid_data, **params)

['9', '10', '11']
[9, 10, 11]


/usr/local/conda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in exp


In [80]:
grid

array([[   inf,    inf,    inf, ...,    inf,    inf,    inf],
       [   inf,    inf,    inf, ...,    inf,    inf,    inf],
       [   inf,    inf,    inf, ...,    inf,    inf,    inf],
       ..., 
       [-2.659, -2.661, -2.663, ..., -2.701, -2.702, -2.704],
       [-2.481, -2.484, -2.487, ..., -2.554, -2.557, -2.561],
       [-2.278, -2.282, -2.287, ..., -2.381, -2.386, -2.391]])

In [ ]:
from tqdm import tqdm
def fit_participants(data):
    
    def pid_mle():
        do = 5
            for i, row in df.iterrows():


    return pd.concat(pid_mle()).reset_index(drop=True)

results = fit_participants(data)